In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, Ridge
import xgboost as xgb
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv('train.csv')
train_data

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,2174,4,16.8,0.0,1.6,53.0,2000.0,0.031,37.0,27.0,21.0
1455,2175,3,10.8,0.0,3.8,45.0,2000.0,0.039,34.0,19.0,20.0
1456,2176,5,18.3,0.0,1.9,54.0,2000.0,0.009,30.0,21.0,22.0
1457,2178,21,20.7,0.0,3.7,37.0,1395.0,0.082,71.0,36.0,216.0


In [3]:
train_data.isnull().sum()

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [4]:
train_data = train_data.drop(labels = ['id', 'hour_bef_humidity', 'hour_bef_pm10', 'hour_bef_pm2.5'], axis = 1)

In [5]:
train_data['hour_bef_temperature'].fillna(train_data['hour_bef_temperature'].median(), inplace = True)
train_data['hour_bef_precipitation'].fillna(train_data['hour_bef_precipitation'].median(), inplace = True)
train_data['hour_bef_windspeed'].fillna(train_data['hour_bef_windspeed'].median(), inplace = True)
train_data['hour_bef_visibility'].fillna(train_data['hour_bef_visibility'].median(), inplace = True)
train_data['hour_bef_ozone'].fillna(train_data['hour_bef_ozone'].median(), inplace = True)

In [6]:
train_data.isnull().sum()

hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_visibility       0
hour_bef_ozone            0
count                     0
dtype: int64

In [7]:
Q1 = train_data['hour_bef_windspeed'].quantile(.25)
Q3 = train_data['hour_bef_windspeed'].quantile(.75)
IQR = Q3 - Q1
max_w = Q3 + (1.5 * Q1)
train_data['hour_bef_windspeed'] = train_data['hour_bef_windspeed'].apply(lambda x: max_w if x > max_w else x)

In [8]:
Q1 = train_data['hour_bef_ozone'].quantile(.25)
Q3 = train_data['hour_bef_ozone'].quantile(.75)
IQR = Q3 - Q1
max_w = Q3 + (1.5 * Q1)
train_data['hour_bef_ozone'] = train_data['hour_bef_ozone'].apply(lambda x: max_w if x > max_w else x)

In [9]:
x_data = train_data.drop(labels = 'count', axis = 1)

In [10]:
# mm = MinMaxScaler().fit(x_data)
# sc_x_data = mm.transform(x_data)

In [11]:
# rs = RobustScaler()
# rs.fit(x_data)
# sc_x_data = rs.transform(x_data)

In [12]:
y_data = train_data['count']
y_data

0        49.0
1       159.0
2        26.0
3        57.0
4       431.0
        ...  
1454     21.0
1455     20.0
1456     22.0
1457    216.0
1458    170.0
Name: count, Length: 1459, dtype: float64

In [13]:
t_x, v_x, t_y, v_y = train_test_split(x_data, y_data, test_size = 0.2, random_state = 22)
t_x.shape, v_x.shape, t_y.shape, v_y.shape

((1167, 6), (292, 6), (1167,), (292,))

### 랜덤 포레스트

In [70]:
params = {
    'n_estimators': [700, 800, 900, 1000],
    'min_samples_split': [10, 20, 30, 40],
    'max_depth' : [5, 10, 15, 20]
}

In [71]:
rf = RandomForestRegressor()

In [72]:
gs = GridSearchCV(rf, param_grid = params, cv = 3, scoring = 'roc_auc').fit(t_x, t_y)

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

...

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 328, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
C:\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\

In [73]:
gs.best_params_

{'max_depth': 5, 'min_samples_split': 10, 'n_estimators': 700}

In [76]:
rf = gs.best_estimator_

In [77]:
rf.fit(t_x, t_y)

RandomForestRegressor(max_depth=5, min_samples_split=10, n_estimators=700)

In [78]:
t_y_pred = rf.predict(t_x)
y_pred = rf.predict(v_x)
v_mse = np.sqrt(mean_squared_error(v_y, y_pred))
t_mse = np.sqrt(mean_squared_error(t_y, t_y_pred))
print(t_mse, v_mse)

36.921567622175075 42.54608466911893


### XGBoost

In [17]:
xgb_m = xgb.XGBRegressor(learning_rate = 0.005, max_depth = 5, n_estimators = 700, subsample = 0.6, min_child_weight = 0.3, random_state = 22)
xgb_m.fit(t_x, t_y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.005, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=0.3,
             missing=nan, monotone_constraints='()', n_estimators=700, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=22,
             reg_alpha=0, reg_lambda=1, ...)

In [18]:
t_y_pred2 = xgb_m.predict(t_x)
y_pred2 = xgb_m.predict(v_x)
v_mse2 = np.sqrt(mean_squared_error(v_y, y_pred2))
t_mse2 = np.sqrt(mean_squared_error(t_y, t_y_pred2))
print(t_mse2, v_mse2)

29.630031534979093 39.100131642162744


### Lasso

In [34]:
lasso = Lasso(alpha = 5, max_iter = 100, random_state = 22).fit(t_x, t_y)
t_y_pred3 = lasso.predict(t_x)
y_pred3 = lasso.predict(v_x)
v_mse3 = np.sqrt(mean_squared_error(v_y, y_pred3))
t_mse3 = np.sqrt(mean_squared_error(t_y, t_y_pred3))
print(t_mse3, v_mse3)

53.502555508385164 55.47338165824577


### Ridge

In [31]:
ridge = Ridge(alpha = 2, max_iter = 30, random_state = 22).fit(t_x, t_y)
t_y_pred4 = ridge.predict(t_x)
y_pred4 = ridge.predict(v_x)
v_mse4 = np.sqrt(mean_squared_error(v_y, y_pred4))
t_mse4 = np.sqrt(mean_squared_error(t_y, t_y_pred4))
print(t_mse4, v_mse4)

52.47431793824676 54.43607744751883


### 테스트

In [20]:
test_data = pd.read_csv('test.csv')
test_data = test_data.drop(labels = ['id', 'hour_bef_humidity', 'hour_bef_pm10', 'hour_bef_pm2.5',], axis = 1)

test_data['hour_bef_temperature'].fillna(test_data['hour_bef_temperature'].median(), inplace = True)
test_data['hour_bef_precipitation'].fillna(test_data['hour_bef_precipitation'].median(), inplace = True)
test_data['hour_bef_windspeed'].fillna(test_data['hour_bef_windspeed'].median(), inplace = True)
test_data['hour_bef_visibility'].fillna(test_data['hour_bef_visibility'].median(), inplace = True)
test_data['hour_bef_ozone'].fillna(test_data['hour_bef_ozone'].median(), inplace = True)

Q1 = test_data['hour_bef_windspeed'].quantile(.25)
Q3 = test_data['hour_bef_windspeed'].quantile(.75)
IQR = Q3 - Q1
max_w = Q3 + (1.5 * Q1)
test_data['hour_bef_windspeed'] = test_data['hour_bef_windspeed'].apply(lambda x: max_w if x > max_w else x)

Q1 = test_data['hour_bef_ozone'].quantile(.25)
Q3 = test_data['hour_bef_ozone'].quantile(.75)
IQR = Q3 - Q1
max_w = Q3 + (1.5 * Q1)
test_data['hour_bef_ozone'] = test_data['hour_bef_ozone'].apply(lambda x: max_w if x > max_w else x)

# mm = MinMaxScaler().fit(test_data)
# sc_test_data = mm.transform(test_data)

In [79]:
pred = rf.predict(test_data)

In [80]:
submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('베이스라인.csv',index = False)